# Second Attempt at Training Script for SegNet 
* This notebook was used ot debug the first attempt
* There were some delays for getting greater quotas
* The training script was found to be the source of the problem
* It has been re-organised and is functional for SegNet2
* SegNet1 experienced some errors in the training jobs, and ther problem has not been resolved

In [1]:
############## Set Up ################

import sagemaker
from sagemaker.pytorch import PyTorch
# ##yan - training on local instance to see what the problem is  
# import boto3
# import os
# from sagemaker.local import LocalSession
# from sagemaker.debugger import rule_configs, DebuggerHookConfig, CollectionConfig

# sagemaker_session = LocalSession()
# sagemaker_session.config = {'local': {'local_code': True}}
# # Make sure to set this to your bucket and location
# BUCKET_NAME = 'sagemaker-firefly-model-artifacts'
# LOCATION_IN_BUCKET = 'smdebug_debug'

# s3_bucket_for_tensors = 's3://{BUCKET_NAME}/{LOCATION_IN_BUCKET}'.format(BUCKET_NAME=BUCKET_NAME, LOCATION_IN_BUCKET=LOCATION_IN_BUCKET)
# ##

sagemaker_session = sagemaker.Session() #use for remote session

bucket = sagemaker_session.default_bucket()
# prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

In [2]:
############# Path Set Up #################
# speciy location of training data
# train_data = 's3://sagemaker-firefly-model-data/Cloud Segmentation Data/38-Cloud_training'.format(bucket, prefix, 'train')
train_data = 's3://sagemaker-firefly-model-data/Cloud Segmentation Data/38-Cloud_training'
# train_data="file:///home/ec2-user/SageMaker/Dataset/CloudSeg/38-Cloud_training"
# validation_data = 's3://{}/{}/{}'.format(bucket, prefix, 'validation') currently training script does this split

# specifiy location for model output to be saved
# s3_output_location = 's3://sagemaker-firefly-model-artifacts'.format(bucket, prefix, 'xgboost_model_sdk')
s3_output_location = 's3://sagemaker-firefly-model-artifacts'

In [3]:
role

'arn:aws:iam::067338613469:role/service-role/AmazonSageMaker-ExecutionRole-20200914T180083'

In [4]:
############# Initialise the PyTorch training estimator object ###########

# train_instance_type='ml.c4.4xlarge'
# estimator = PyTorch(entry_point='segnet2_train_script.py',
#                     role=role,
#                     framework_version='1.4.0',
#                     train_instance_count=1,
#                     train_instance_type='local',
#                     output_path=s3_output_location,
#                     hyperparameters={
#                         'epochs': 5,
#                         'learning_rate':0.001,
#                         'batch-size':8
#                     },
#                     debugger_hook_config = DebuggerHookConfig(
#                         s3_output_path=s3_bucket_for_tensors,  # Required
#                         collection_configs=[
#                             CollectionConfig(
#                                 name="conv0_tensors",
#                                 parameters={
#                                     "include_regex": "*",
#                                     "save_interval": "100"
#                                 }
#                             )
#                         ]
#                     )
#                    )
estimator = PyTorch(entry_point='Refactored_segnet2_train_script.py',
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=2,
                    train_instance_type='ml.p2.xlarge',
                    output_path=s3_output_location,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': 5,
                        'learning_rate':0.001,
                        'backend': 'gloo',
                        'batch_size':8
                    }
                   )
# add train_use_spot_instances = True for spot training

In [5]:
############## Run the training ################
# estimator.fit({'train': train_data})
estimator.fit({'train': train_data}, wait=False, logs='All') 

\#job runs in background with wait=False

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


trying to get some debugging code in here

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_data(trial, tname, batch_index, steps_range, mode=modes.GLOBAL):
    tensor = trial.tensor(tname)
    vals = []
    for s in steps_range:
        val = tensor.value(step_num=s, mode=mode)[batch_index][0]
        vals.append(val)
    return vals

def create_plots(steps_range):
    fig, axs = plt.subplots(nrows=1, ncols=len(steps_range), constrained_layout=True, figsize=(2*len(steps_range), 2),
                            subplot_kw={'xticks': [], 'yticks': []})
    return fig, axs

def plot_tensors(trial, layer, batch_index, steps_range):
    if len(steps_range) > 0:    
        fig, axs = create_plots(steps_range)
        vals = get_data(trial, layer, batch_index, steps_range)

        for ax, image, step in zip(axs.flat if isinstance(axs, np.ndarray) else np.array([axs]), vals, steps_range):
            ax.imshow(image, cmap='gray')
            ax.set_title(str(step))
        plt.show()